In [1]:
from nwm_reanalysis import NWM
from dask.distributed import Client, LocalCluster

In [2]:
start_date = "2010-01-01"
end_date = "2010-12-31"
comids = [2043493]

nwm = NWM(start_date=start_date, end_date=end_date, comids=comids)
scheduler = LocalCluster(n_workers=4, threads_per_worker=4, processes=False)
scheduler

LocalCluster(1b4bb64d, 'inproc://10.144.103.181/54604/1', workers=4, threads=16, memory=127.76 GiB)

In [3]:
%%time
# %%prun
nwm.request_timeseries(scheduler=scheduler, optimize=False)
nwm.data

CPU times: total: 1min 17s
Wall time: 4min 44s


<xarray.Dataset>
Dimensions:     (time: 8784, feature_id: 1)
Coordinates:
  * feature_id  (feature_id) int32 2043493
    latitude    (feature_id) float32 34.43
    longitude   (feature_id) float32 -83.74
  * time        (time) datetime64[ns] 2010-01-01 ... 2011-01-01T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 1.27 1.26 1.25 1.24 ... 7.6 5.6 3.5
    velocity    (time, feature_id) float64 0.56 0.56 0.56 ... 1.09 1.03 0.78
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

In [4]:
df = nwm.set_output(return_dataframe=False)
# df

(Timestamp('2010-01-01 00:00:00'), 2043493)


AttributeError: 'int' object has no attribute 'strftime'

In [ ]:
%%time
nwm.request_timeseries(scheduler=scheduler, optimize=False)

In [8]:
import xarray as xr
import fsspec
import dask

In [9]:
nwm_url = "s3://noaa-nwm-retro-v2-zarr-pds"

In [15]:
%%time
drop_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral"]
ds = xr.open_zarr(fsspec.get_mapper(nwm_url, anon=True), consolidated=True, drop_variables=drop_variables)
# ds = ds.chunk(chunks='auto')

Wall time: 11.3 s


In [ ]:
%%time
variables = ["streamflow", "velocity"]
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
# with dask.config.set(**{'array.slicing.split_large_chunks': True}):
ds_data = ds[variables].sel(feature_id=nwm.comids).sel(time=slice(start_string, end_string)).load(optimize_graph=False, traverse=False)
data = ds_data
print(f"Start: {start_string}, End: {end_string}")
data

In [15]:
%%time
start_string = f"{nwm.start_date.year}-{nwm.start_date.month}-{nwm.start_date.day}"
end_string = f"{nwm.end_date.year}-{nwm.end_date.month}-{nwm.end_date.day}"
ds_data = ds.sel(feature_id=nwm.comids)
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
ds_data

Wall time: 1min 34s


<xarray.Dataset>
Dimensions:     (feature_id: 1, time: 8784)
Coordinates:
  * feature_id  (feature_id) int32 2043493
    latitude    (feature_id) float32 34.43
    longitude   (feature_id) float32 nan
  * time        (time) datetime64[ns] 2010-01-01 ... 2011-01-01T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 1.27 1.26 1.25 1.24 ... 7.6 5.6 3.5
    velocity    (time, feature_id) float64 0.56 0.56 0.56 ... 1.09 1.03 0.78
Attributes: (12/17)
    Conventions:                CF-1.6
    cdm_datatype:               Station
    code_version:               v5.1.0-alpha11
    dev:                        dev_ prefix indicates development/internal me...
    dev_NOAH_TIMESTEP:          3600
    dev_OVRTSWCRT:              1
    ...                         ...
    model_output_type:          channel_rt
    model_output_valid_time:    2018-12-28_00:00:00
    model_total_valid_times:    2208
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...
    station_dimension:          feature_id
    stream_order_output:        1

In [23]:
ds_data = ds_data.sel(time=slice(start_string, end_string)).load(optimize_graph=True, traverse=False).load()
print(f"Start: {start_string}, End: {end_string}")
ds_data

,,streamflow,velocity,latitude,longitude
feature_id,time,,,,


In [1]:
all_variables = ["elevation", "order", "qBtmVertRunoff", "qBucket", "qSfcLatRunoff", "q_lateral", "streamflow", "velocity"]
d_v = list(set(all_variables) - set(variables))
d_v

NameError: name 'variables' is not defined